In [1]:
import sys
from pathlib import Path
project_root = Path.cwd().parent.absolute() # get project root path for loading modules in notebook
sys.path.insert(0, str(project_root))


import torch
import torch.nn as nn
import torch.nn.functional as F

import os
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
from functools import reduce, partial
from timeit import default_timer

from src.utils.utils import *
from src.models.base import FNO2d
from src.models.multi_step import BOON_FNO2d

/opt/conda/envs/bert/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.manual_seed(0)
np.random.seed(0)

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Dirichlet
1D Stokes' Equation with Dirichlet boundary conditions for $M=25$ multi-step step prediction at $t_M= 1.2$.

In [4]:
ntrain = 1000
ntest = 200

sub = 1 # subsampling rate
N = 500 // sub # total grid size divided by the subsampling rate

s = N

# Set NN training parameters
batch_size = 20
learning_rate = 0.001

epochs = 500
scheduler_step = 50
scheduler_gamma = 0.5

# Set FNO parameters
modes = 12
width = 32

# 1 input time of the initial condition
T_in = 1
# number of multi-steps to predict
T = 25

In [ ]:
################################################################
# read data
################################################################
downloader = DataDownloader()
# Use downloader.download(id = "stokes_2D", tag = "nu_0_point_001") if only downloading a single file 
downloader.download(id = "Stokes_Dir_2D")
rw_ = downloader.locate(id = "Stokes_Dir_2D", tag = "nu_0_point_1")

# LOAD YOUR DATA HERE consists of pairs (a,u) with randomly generated initial conditions or PDE parameters a
# and solution u
data_a = rw_['a'].astype(np.float32) # shape (num_random_simulations, number_grid_points, 1)
data_u = rw_['u'].astype(np.float32) # shape (num_random_simulations, number_grid_points, num_time_steps)


x_train = data_a[:ntrain,::sub,:T_in]
x_train = x_train.astype(np.float32)
x_train = torch.from_numpy(x_train)

y_train = data_u[:ntrain,::sub,-T:]
y_train = y_train.astype(np.float32)
y_train = torch.from_numpy(y_train)

x_test = data_a[-ntest:,::sub,:T_in]
x_test = x_test.astype(np.float32)
x_test = torch.from_numpy(x_test)

y_test = data_u[-ntest:,::sub,-T:]
y_test = y_test.astype(np.float32)
y_test = torch.from_numpy(y_test)

In [7]:
x_train.shape, y_train.shape

(torch.Size([1000, 500, 1]), torch.Size([1000, 500, 25]))

In [ ]:
x_test.shape, y_test.shape

In [ ]:
x_train = x_train.reshape(ntrain,s,1,T_in).repeat([1,1,T,1])
print(x_train.shape)
x_test = x_test.reshape(ntest,s,1,T_in).repeat([1,1,T,1])

In [9]:
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, y_train), 
                                           batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test), 
                                          batch_size=batch_size, shuffle=False)

In [10]:
# Initialize FNO using modes and width parameter
base_no = FNO2d(modes, modes, width)
model = BOON_FNO2d(width, 
                    base_no,
                    bdy_type='dirichlet').to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=scheduler_step, gamma=scheduler_gamma)

myloss = LpLoss(size_average=False)

for ep in range(epochs):
    model.train()
    t1 = default_timer()
    train_l2 = 0
    for x, y in train_loader:
        bs, nx, T, _ = x.shape
        x, y = x.to(device), y.to(device)
        
        optimizer.zero_grad()
        
        bdy_left = y[:, 0, :].reshape(bs, 1, T) # add extra dimension to take care of 
                             #  model channel structure
        bdy_right = y[:, -1, :].reshape(bs, 1, T)
        
        out = model(x, 
                    bdy_left={'val':bdy_left}, 
                    bdy_right={'val':bdy_right}
                   ).view(bs, s, T)

        l2 = myloss(out.view(bs, -1), y.view(bs, -1))
        l2.backward() # use the l2 relative loss

        optimizer.step()
        train_l2 += l2.item()
    scheduler.step()

    model.eval()
    test_l2 = 0.0
    with torch.no_grad():
        for x, y in test_loader:
            bs, nx, T, _ = x.shape
            x, y  = x.to(device), y.to(device)
            bdy_left = y[:, 0, :].reshape(bs, 1, T) # add extra dimension to take care of 
                                                    #  model channel structure
            bdy_right = y[:, -1, :].reshape(bs, 1, T)

            out = model(x, 
                        bdy_left={'val':bdy_left}, 
                        bdy_right={'val':bdy_right}
                       ).view(bs, s, T)

            test_l2 += myloss(out.view(bs, -1), y.view(bs, -1)).item()

    train_l2 /= ntrain
    test_l2 /= ntest

    t2 = default_timer()
    print(ep, t2-t1,  train_l2, test_l2)

0 13.17493495112285 1.112001835823059 0.7989439296722413
1 12.063697924371809 0.6866058821678162 0.44218212604522705
2 12.059206295758486 0.2355163459777832 0.4333240246772766
3 12.05852381605655 0.1408351458311081 0.24416667342185974
4 12.075867711100727 0.09210876071453095 0.14577823698520662
5 12.07484277477488 0.059072255611419676 0.09938824355602265
6 12.076462741009891 0.06629376035928726 0.16725133419036864
7 12.082855060696602 0.048677261024713515 0.12072305083274841
8 12.055011734832078 0.0344178723692894 0.1133331573009491
9 12.065285377204418 0.03281695699691772 0.1273738783597946
10 12.061399090103805 0.05431474763154984 0.09218336015939713
11 12.06283085187897 0.03341848412156105 0.16171648144721984
12 12.064962890930474 0.04716404202580452 0.08603286117315292
13 12.0506935371086 0.027574355155229567 0.0621413579583168
14 12.07439676579088 0.04012840616703033 0.09946856051683425
15 12.098097985144705 0.045425780773162845 0.09726897120475769
16 12.064284251071513 0.03580886

# Neumann
1D Heat Equation with Neumann boundary conditions for $M=25$ multi-step prediction at $t_M= 2$ with initial condition $u_0(x) = \cos(\omega \pi x)$, where $\omega$ is uniformly sampled in $[2.01, 3.99]$ with fixed $k=0.01$ and $U=5$.

In [83]:
ntrain = 1000
ntest = 200

sub = 2 #subsampling rate
N = 2048 // sub # total grid size divided by the subsampling rate

s = N

# Set NN training parameters
batch_size = 20
learning_rate = 0.001

epochs = 500
scheduler_step = 50
scheduler_gamma = 0.5 # what is gamma?

# Set FNO parameters
modes = 12
width = 32

# 1 input time of the initial condition
T_in = 1
# number of multi-steps to predict
T = 25

In [ ]:
################################################################
# read data
################################################################


downloader = DataDownloader()
downloader.download(id = "Heat_Neu_2D")
rw_ = downloader.locate(id = "Heat_Neu_2D", tag = "neumann")

# LOAD YOUR DATA HERE consists of pairs (a,u) with randomly generated initial conditions or PDE parameters a
# and solution u
data_a = rw_['a'].astype(np.float32) # shape (num_random_simulations, number_grid_points, 1)
data_u = rw_['u'].astype(np.float32) # shape (num_random_simulations, number_grid_points, num_time_steps)

x_train = data_a[:ntrain,::sub,:T_in]
x_train = x_train.astype(np.float32)
x_train = torch.from_numpy(x_train)

y_train = data_u[:ntrain,::sub,-T:]
y_train = y_train.astype(np.float32)
y_train = torch.from_numpy(y_train)

x_test = data_a[-ntest:,::sub,:T_in]
x_test = x_test.astype(np.float32)
x_test = torch.from_numpy(x_test)

y_test = data_u[-ntest:,::sub, -T:]
y_test = y_test.astype(np.float32)
y_test = torch.from_numpy(y_test)

In [11]:
x_train.shape, y_train.shape

(torch.Size([1000, 2048, 1]), torch.Size([1000, 2048, 25]))

In [86]:
x_test.shape, y_test.shape

(torch.Size([200, 1024, 1]), torch.Size([200, 1024, 25]))

In [92]:
x_train = x_train.reshape(ntrain,s,1,T_in).repeat([1,1,T,1])
print(x_train.shape)
x_test = x_test.reshape(ntest,s,1,T_in).repeat([1,1,T,1])

torch.Size([1000, 1024, 25, 1])


In [94]:
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, y_train), 
                                            batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test), 
                                            batch_size=batch_size, shuffle=False)

In [88]:
lb = 0
ub = 1

h = (ub-lb)/(N-1) # grid spacing for finite difference

# 4th order scheme
coeffs_finite_difference_right = np.array([-1/(3*h), 3/(2*h), -3/h , 11/(6*h)])
coeffs_finite_difference_right = coeffs_finite_difference_right.astype(np.float32)

normalized_coeff_right = np.array([-2/11, 9/11, -18/11, 6*h/11]).astype(np.float32)
normalized_coeff_right = torch.from_numpy(normalized_coeff_right).to(device)

diff_fn_right = partial(compute_finite_diff, normalized_coeff_right, loc=-1)

# First order scheme
coeffs_finite_difference_left = np.array([1, -1]) 
coeffs_finite_difference_left = coeffs_finite_difference_left.astype(np.float32)

normalized_coeff_left = np.array([1, -1]).astype(np.float32)
normalized_coeff_left = torch.from_numpy(normalized_coeff_left).to(device)
diff_fn_left = partial(compute_finite_diff, normalized_coeff_left, loc=0)


U = np.array([5])
U = torch.from_numpy(U).to(device)

t_final = np.array([0.5])
num_dt = 200
t = np.linspace(0, t_final, num_dt)
t = t[-T:]
t = torch.from_numpy(t).cuda()
neumann_bdy_left = 0.0 * t
neumann_bdy_right = U*torch.sin(torch.pi*t)

In [89]:
base_no = FNO2d(modes, modes, width)
model = BOON_FNO2d(width, 
                    base_no,
                    bdy_type='neumann').to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=scheduler_step, gamma=scheduler_gamma)

myloss = LpLoss(size_average=False)

for ep in range(epochs):
    model.train()
    t1 = default_timer()
    train_l2 = 0
    for x,y in train_loader:
        bs, nx, T, _ = x.shape
        x, y = x.to(device), y.to(device)
        
        optimizer.zero_grad()
        out = model(x, 
                    bdy_left={'val':neumann_bdy_left.reshape(1,1,T).repeat(bs, 1, 1), 'diff_fn':diff_fn_left},
                    bdy_right={'val':neumann_bdy_right.reshape(1,1,T).repeat(bs, 1, 1), 'diff_fn':diff_fn_right},
                   ).reshape(bs, s, T)

        l2 = myloss(out.view(bs, -1), y.view(bs, -1))
        l2.backward() # use the l2 relative loss

        optimizer.step()
        train_l2 += l2.item()
        

    scheduler.step()
    model.eval()
    test_l2 = 0.0
    with torch.no_grad():
        for x, y in test_loader:
            bs, nx, T, _ = x.shape
            x, y = x.to(device), y.to(device)

            out = model(x, 
                        bdy_left={'val':neumann_bdy_left.reshape(1,1,T).repeat(bs, 1, 1), 'diff_fn':diff_fn_left},
                        bdy_right={'val':neumann_bdy_right.reshape(1,1,T).repeat(bs, 1, 1), 'diff_fn':diff_fn_right},
                       ).reshape(bs, s, T)

            test_l2 += myloss(out.view(bs, -1), y.view(bs, -1)).item()

    # train_mse /= len(train_loader)
    train_l2 /= ntrain
    test_l2 /= ntest

    t2 = default_timer()
    print(ep, t2-t1, train_l2, test_l2)

0 13.56806010287255 0.4697633533477783 0.3638677787780762
1 13.49183622887358 0.21164067840576173 0.34757102251052857
2 13.468005978967994 0.11115208768844605 0.34591084241867065
3 13.469817613717169 0.0752148288488388 0.16458994030952454
4 13.494669828098267 0.051995068192481994 0.13785767376422883
5 13.473367569968104 0.05098898708820343 0.15851377546787263
6 13.503576235845685 0.05543627405166626 0.11153204321861267
7 13.511253986041993 0.04290539216995239 0.09297127723693847
8 13.512618033681065 0.03941105821728706 0.09539728760719299
9 13.568665483035147 0.028695187717676162 0.12533587574958802
10 13.553872446063906 0.037333181977272034 0.12751367568969726
11 13.556182018946856 0.04462537959218025 0.07649616837501526
12 13.55066985078156 0.029647189676761627 0.09485438764095307
13 13.570563718210906 0.023593211680650712 0.06637237042188644
14 13.580372001044452 0.030934628129005434 0.08804313868284225
15 13.554088388103992 0.029274197041988373 0.08205933034420014
16 13.55765641899

In [36]:
train_l2, test_l2 # final test and train loss

(0, 0.02145304039120674)

# Periodic
Burgers' Equation with periodic BC with random initial condition $u_0(x) \sim \mathcal{N}(0, 625(-\Delta + 25 I)^{-2})$

In [ ]:
################################################################
# read data
################################################################

downloader = DataDownloader()
downloader.download(id = "Burgers_Per_2D")
rw_ = downloader.locate(id = "Burgers_Per_2D", tag = "pino")

################################################################
#  configurations
################################################################
ntrain = 1000
ntest = 200

sub = 1
N = 128

s = N

batch_size = 20
learning_rate = 0.001

epochs = 500
scheduler_step = 100
scheduler_gamma = 0.5

modes = 12
width = 32

T_in = 1

T = 25

In [13]:
################################################################
# read data
################################################################

# LOAD YOUR DATA HERE consists of pairs (a,u) with randomly generated initial conditions or PDE parameters a
# and solution u
data_a = rw_['input'].astype(np.float32) # shape (num_random_simulations, number_grid_points, 1)
data_u = rw_['output'].astype(np.float32) # shape (num_random_simulations, number_grid_points, num_time_steps)

x_train = data_a[:ntrain,::sub]
x_train = x_train.astype(np.float32)
x_train = torch.from_numpy(x_train).unsqueeze(-1)

y_train = data_u[:ntrain,::sub,-T:]
y_train = y_train.astype(np.float32)
y_train = torch.from_numpy(y_train)

x_test = data_a[-ntest:,::sub]
x_test = x_test.astype(np.float32)
x_test = torch.from_numpy(x_test).unsqueeze(-1)

y_test = data_u[-ntest:,::sub, -T:]
y_test = y_test.astype(np.float32)
y_test = torch.from_numpy(y_test)

In [14]:
x_train.shape, y_train.shape

(torch.Size([1000, 128, 1]), torch.Size([1000, 128, 25]))

In [7]:
x_train = x_train.reshape(ntrain,s,1,T_in).repeat([1,1,T,1])
print(x_train.shape)
x_test = x_test.reshape(ntest,s,1,T_in).repeat([1,1,T,1])

torch.Size([1000, 128, 25, 1])


In [8]:
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, y_train), 
                                            batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test), 
                                            batch_size=batch_size, shuffle=False)

In [9]:
base_no = FNO2d(modes, modes, width)
model = BOON_FNO2d(width, 
                    base_no,
                    bdy_type='periodic').to(device)

In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=scheduler_step, gamma=scheduler_gamma)

myloss = LpLoss(size_average=False)

for ep in range(epochs):
    model.train()
    t1 = default_timer()
    train_l2 = 0
    for x, y in train_loader:
        bs, nx, T = y.shape
        x, y = x.to(device), y.to(device)
        
        optimizer.zero_grad()
        out = model(x).reshape(bs, s, T)

        l2 = myloss(out.view(bs, -1), y.view(bs, -1))
        l2.backward() # use the l2 relative loss

        optimizer.step()
        train_l2 += l2.item()
        

    scheduler.step()
    model.eval()
    test_l2 = 0.0
    test_l2_near_bdy = 0.0
    with torch.no_grad():
        for x, y in test_loader:
            bs, nx, T, _ = x.shape
            x, y = x.to(device), y.to(device)
            out = model(x)
            test_l2 += myloss(out.view(bs, -1), y.view(bs, -1)).item()

    train_l2 /= ntrain
    test_l2 /= ntest

    t2 = default_timer()
    print(ep, t2-t1, train_l2, test_l2)

0 2.423506321851164 0.5993650617599487 0.23130704164505006
1 1.4333614851348102 0.17034529089927675 0.11094810247421265
2 1.4287836370058358 0.09397487938404084 0.07681365728378296
3 1.428499094210565 0.06791800278425217 0.049483804404735564
4 1.4308553016744554 0.03857289746403694 0.04084377706050873
5 1.4301004828885198 0.033280477315187455 0.027905315458774567
6 1.431079301983118 0.032243052065372466 0.038328715562820435
7 1.4306932059116662 0.03725769686698913 0.026067704111337662
8 1.4269019598141313 0.025415767058730124 0.026541863679885865
9 1.428836372680962 0.027038739442825317 0.016362170726060866
10 1.4238203996792436 0.022707321584224702 0.016347936540842056
11 1.4300414370372891 0.0189953845590353 0.0378798297047615
12 1.4365873537026346 0.031204399541020392 0.02082627937197685
13 1.4195692613720894 0.024537304431200026 0.019124159067869188
14 1.427011493127793 0.026212203204631804 0.02099927335977554
15 1.4319462580606341 0.02363977198302746 0.011298475712537765
16 1.4509